In [2]:
import codecs
import re
import nltk
import collections
import numpy as np
import subprocess
import json
import multiprocessing
import os
import gc
import sklearn
import networkx as nx
import scipy
import metric_learn
import itertools
import pandas as pd
import pandas
from IPython.display import display, HTML
from ipywidgets import FloatProgress

In [31]:
%run shared.py

In [25]:
import sqlite3
sql_conn = sqlite3.connect('books.db')
c = sql_conn.cursor()

In [50]:
def try_drop_table(name):
    try:
        c.execute("drop table %s;" % name)
    except:
        print("no table %s" % name)

In [75]:
try_drop_table("authors")
try_drop_table("books")
try_drop_table("words")

In [39]:
try_drop_table("chunks_50000")
try_drop_table("chunks_100000")

no table chunks_50000
no table chunks_100000


In [32]:
def char_range(start, end):
    return [chr(char) for char in range(ord(start), ord(end) + 1)]


In [76]:
c.execute("""
    create table authors (
        id integer primary key,
        name text
    );""")
c.execute("""
    
    create table books (
        id integer primary key,
        author integer,
        name text
    );
""")
c.execute("""
    
    create table words (
        id integer primary key,
        sent integer,
        book integer,
        text text,
        lex text,
        wt double,
        pos text,
        gr text
    );
""")
c.execute("""
    create index if not exists index_sent ON words (sent);
""")
c.execute("""
    create index if not exists index_book ON words (book);
""")
sql_conn.commit()

In [69]:
books_descr = pd.read_csv("selected_books.csv")
books_descr = books_descr[(books_descr.downloaded & ~books_descr.poem_or_play)]

[(0, Unnamed: 0                                                          0
  index                                                               0
  cat_collection                                             Пряслины 1
  cat_comment                                                       NaN
  cat_f                                                         Абрамов
  cat_i                                                           Федор
  cat_id                                                         158030
  cat_issue_year                                                   1982
  cat_lang                                                           ru
  cat_o                                                   Александрович
  cat_title                                             Братья и сестры
  mukhin_fio                                Абрамов Федор Александрович
  mukhin_libru                             http://lib.ru/PROZA/ABRAMOW/
  mukhin_title                                          Брать

In [77]:
def get_json_from_file(file):
    with codecs.open(file, encoding='utf-8') as f:
        content = f.read()
        return [json.loads(l) for l in content.splitlines()]
    
def get_rows(word_analysis):
    sentence = 1
    for i, w in enumerate(word_analysis):
        #print(sentence)
        if w['text'] == '\\s':
            sentence += 1
        else:
            analysis = w.get('analysis')
            if analysis:
                a = analysis[0]
                lex = a['lex']
                wt = a['wt']
                gr = a['gr']
                pos = re.split('[,=()]', gr)[0]
                yield (sentence, w['text'], lex, wt, pos, gr)
            else:
                yield (sentence, w['text'], None, None, None, None)

for i, row in progressify_list(list(books_descr.iterrows())):
    book_id = row.cat_id
    book_author = row.fi
    book_name = row.cat_title
    print(book_id, book_author, book_name)
    author_id_row = c.execute("select id from authors where name = ?", (book_author,)).fetchone()
    author_id = author_id_row[0] if author_id_row else c.execute(
        "insert into authors (name) values (?)", (book_author,)).lastrowid
    book_id = c.execute("insert into books (id, name, author) values (?, ?, ?)", (book_id, book_name, author_id)).lastrowid
    word_analysis = get_json_from_file("json/{}.fb2.txt.json".format(book_id))
    rows = list(get_rows(word_analysis))
    c.executemany("""insert into words (sent, text, lex, wt, pos, gr, book)
                  values (?, ?, ?, ?, ?, ?, %s)""" % book_id, rows)
sql_conn.commit()

158030 Абрамов Федор Братья и сестры
158033 Абрамов Федор Две зимы и три лета
158032 Абрамов Федор Дом
158036 Абрамов Федор Пути-перепутья
159231 Аверченко Аркадий Подходцев и двое других
417321 Аверченко Аркадий Разумная экономия
159241 Аверченко Аркадий Шутка мецената
150859 Айтматов Чингиз Белый пароход
65797 Айтматов Чингиз Пегий пес, бегущий краем моря
65796 Айтматов Чингиз Плаха
131819 Акунин Борис Алтын-Толобас
144729 Акунин Борис Нефритовые четки
461230 Акунин Борис Статский советник
305713 Акунин Борис Турецкий гамбит
191882 Акунин Борис Ф. М.
426019 Алексиевич Светлана Время секонд хэнд
70938 Алексиевич Светлана У войны - не женское лицо…
427115 Алексиевич Светлана Цинковые мальчики
347944 Алексин Анатолий Безумная Евдокия
122412 Алексин Анатолий Здоровые и больные
1061 Алексин Анатолий Мой брат играет на кларнете
1068 Алексин Анатолий Поздний ребенок
1070 Алексин Анатолий Раздел имущества
141811 Алешковский Петр Институт сновидений
435044 Алешковский Петр Крепость
1122 Алешк

69300 Мамин-Сибиряк Дмитрий Хлеб
35517 Мандельштам Осип Четвертая проза
268404 Мандельштам Осип Шум времени
380537 Матвеева Анна Небеса
36282 Матвеева Анна Перевал Дятлова
101185 Можаев Борис Живой
38222 Можаев Борис История села Брехова, писанная Петром Афанасиевичем Булкиным
173143 Можаев Борис Мужики и бабы
173145 Можаев Борис Полтора квадратных метра
119564 Набоков Владимир Дар
38566 Набоков Владимир Другие берега
217496 Набоков Владимир Защита Лужина
385742 Набоков Владимир Король, дама, валет
232264 Набоков Владимир Лолита
76082 Набоков Владимир Машенька
322391 Набоков Владимир Приглашение на казнь
200025 Нагибин Юрий Тьма в конце туннеля
331966 Нагибин Юрий Чистые пруды
343914 Некрасов Виктор В окопах Сталинграда
98267 Некрасов Виктор В родном городе
376834 Некрасов Виктор Вторая ночь
98261 Некрасов Виктор Кира Георгиевна
98266 Некрасов Виктор Судак
267779 Носов Николай Весёлая семейка
287992 Носов Николай Незнайка в Солнечном городе
287990 Носов Николай Незнайка на Луне
287996 

186062 Сологуб Федор Капли крови


In [81]:
def get_chunks(book_id, chunk_size):
    words = list(c.execute("select id, text, lex, pos, gr, wt, sent from words where book = ?", (book_id,)))
    df = pandas.DataFrame(words, columns='id, text, lex, pos, gr, wt, sent'.split(', '))
    df['len'] = df.text.str.len()
    df = df[(df.len < 50) & (df.sent > 1) & (df.sent < df.iloc[-1].sent)]
    #df['cum_len'] = df.len.cumsum()
    sent_len = df.groupby('sent').len.sum().to_frame().len.cumsum().to_frame()
    sent_len['chunk'] = sent_len.len // chunk_size
    df['chunk'] = df.sent.map(sent_len.chunk)
    df = df.set_index(['chunk', 'sent'])
    chunks = df.index.get_level_values('chunk').unique()
    return [df.loc[ch,] for ch in chunks]

def write_chunk_boundaries(chunk_size):
    table = 'chunks_%s' % chunk_size
    print(table)
    try_drop_table(table)
    c.execute("""
    
    create table %s (
        id integer primary key,
        begin integer,
        end integer
    );
    """ % table)
    for book_id, in progressify_list(list(c.execute("select id from books"))):
        chunks = get_chunks(book_id, chunk_size)
        c.executemany("""insert into %s (begin, end)
                  values (?, ?)""" % table, [(int(c.id.iloc[0]), int(c.id.iloc[-1])) for c in chunks])
    sql_conn.commit()

write_chunk_boundaries(50000)

chunks_50000


In [82]:
write_chunk_boundaries(100000)

chunks_100000
no table chunks_100000


In [83]:
sql_conn.commit()

In [5]:
list(c.execute("select * from words where book = '231' limit 1000"))

[(35775951, 1, 231, 'child', None, None, None, None),
 (35775952, 1, 231, '_', None, None, None, None),
 (35775953, 1, 231, 'prose', None, None, None, None),
 (35775954, 1, 231, ' ', None, None, None, None),
 (35775955, 1, 231, 'Григорий', 'григорий', 0.0, 'S', 'S,имя,муж,од=им,ед'),
 (35775956, 1, 231, ' ', None, None, None, None),
 (35775957,
  1,
  231,
  'Бенционович',
  'бенционович',
  0.0,
  'S',
  'S,отч,муж,од=им,ед'),
 (35775958, 1, 231, ' ', None, None, None, None),
 (35775959, 1, 231, 'Остер', 'острый', 87.2, 'A', 'A=ед,кр,муж'),
 (35775960, 1, 231, ' ', None, None, None, None),
 (35775961, 1, 231, 'Сказка', 'сказка', 53.9, 'S', 'S,жен,неод=им,ед'),
 (35775962, 1, 231, ' ', None, None, None, None),
 (35775963, 1, 231, 'с', 'с', 11151.7, 'PR', 'PR='),
 (35775964, 1, 231, ' ', None, None, None, None),
 (35775965,
  1,
  231,
  'подробностями',
  'подробность',
  44.7,
  'S',
  'S,жен,неод=твор,мн'),
 (35775966, 1, 231, '\n', None, None, None, None),
 (35775967, 1, 231, '\n', 

In [84]:
sql_conn.close()